## Frequency_tables_wide_format

#### Import libraries

In [28]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
import geopandas as gpd
from sqlalchemy import create_engine
from sqlalchemy import text
from shapely.geometry import LineString

#### Create the connection with the DB (optional)

In [3]:
# Step 1: Create the connection with the DB
engine = create_engine("postgresql://urbaninfo:@cirrus.ita.chalmers.se/se_tuptp")
conn = engine.connect()

In [4]:
# Step 2: Create a new schema when it is necessary
#schema = '''CREATE SCHEMA name_schema'''
#conn.execute(schema)

##### To create this table format, it is necessary to use the nodes generated in the Frequency_long_format_tables.

### Links

In [55]:
# Step 1: Import data
# In this example, we are utilizing data from Västra Götaland, although these can be modified based on specific requirements.
# nodes_pt_0
nodes_pt_0_query = text('SELECT * FROM pt_0_baseline.nodes_pt_0')
nodes_pt_0 = gpd.read_postgis(nodes_pt_0_query, con=engine.connect(), geom_col='geometry')

#links pt_0
links_pt_0_query = text('SELECT * FROM pt_0_baseline.links_pt_0')
links_pt_0 = gpd.read_postgis(links_pt_0_query, con=engine.connect(), geom_col='geometry')

# nodes_pt_1
nodes_pt_1_query = text('SELECT * FROM pt_1_mdirections_mstops_mroutes.nodes_pt_1')
nodes_pt_1 = gpd.read_postgis(nodes_pt_1_query, con=engine.connect(), geom_col='geometry')

#links pt_1
links_pt_1_long_query = text('SELECT * FROM pt_1_mdirections_mstops_mroutes.links_pt_1_long')
links_pt_1_long = gpd.read_postgis(links_pt_1_long_query, con=engine.connect(), geom_col='geometry')

In [11]:
# Proprocess the Data
# Step 2: Define combinations of day and time periods for frequency columns
day_time_combinations = [
    ('Monday', 'morning'), ('Tuesday', 'morning'), ('Wednesday', 'morning'), ('Thursday', 'morning'), ('Friday', 'morning'), ('Saturday', 'morning'), ('Sunday', 'morning'),
    ('Monday', 'peak_morning'), ('Tuesday', 'peak_morning'), ('Wednesday', 'peak_morning'), ('Thursday', 'peak_morning'), ('Friday', 'peak_morning'), ('Saturday', 'peak_morning'), ('Sunday', 'peak_morning'),
    ('Monday', 'afternoon'), ('Tuesday', 'afternoon'), ('Wednesday', 'afternoon'), ('Thursday', 'afternoon'), ('Friday', 'afternoon'), ('Saturday', 'afternoon'), ('Sunday', 'afternoon'),
    ('Monday', 'peak_evening'), ('Tuesday', 'peak_evening'), ('Wednesday', 'peak_evening'), ('Thursday', 'peak_evening'), ('Friday', 'peak_evening'), ('Saturday', 'peak_evening'), ('Sunday', 'peak_evening'),
    ('Monday', 'evening'), ('Tuesday', 'evening'), ('Wednesday', 'evening'), ('Thursday', 'evening'), ('Friday', 'evening'), ('Saturday', 'evening'), ('Sunday', 'evening'),
    ('Monday', 'night'), ('Tuesday', 'night'), ('Wednesday', 'night'), ('Thursday', 'night'), ('Friday', 'night'), ('Saturday', 'night'), ('Sunday', 'night'),
    ('Monday', 'night(dawn)'), ('Tuesday', 'night(dawn)'), ('Wednesday', 'night(dawn)'), ('Thursday', 'night(dawn)'), ('Friday', 'night(dawn)'), ('Saturday', 'night(dawn)'), ('Sunday', 'night(dawn)')
]

# Step 3: Assign frequencies based on day and time period combinations
for day, time_period in day_time_combinations:
    column_name = f'freq_{day.lower()}_{time_period.lower()}'
    mask = (links_pt_1_long['day_week'] == day) & (links_pt_1_long['time_period'] == time_period)
    links_pt_1_long.loc[mask, column_name] = links_pt_1_long['frequency_dayweek']

# Step 4 (optional): Display the results
links_pt_1_long.head()

,source,target,time_distance,stop_id_source,stop_id_target,node_id_source,node_id_target,route_id,route_short_name,stop_name_source,...,freq_friday_night,freq_saturday_night,freq_sunday_night,freq_monday_night(dawn),freq_tuesday_night(dawn),freq_wednesday_night(dawn),freq_thursday_night(dawn),freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn)
0,9022014001004001_9011014502300000,9022014003960006_9011014502300000,85.0,9022014001004001,9022014003960006,9022014001004001,9022014003960006,9011014502300000,23,Amhult Resecentrum,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9022014001004001_9011014502300000,9022014003960006_9011014502300000,102.0,9022014001004001,9022014003960006,9022014001004001,9022014003960006,9011014502300000,23,Amhult Resecentrum,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9022014001004001_9011014502300000,9022014003960006_9011014502300000,85.0,9022014001004001,9022014003960006,9022014001004001,9022014003960006,9011014502300000,23,Amhult Resecentrum,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9022014001004001_9011014502300000,9022014003960006_9011014502300000,85.0,9022014001004001,9022014003960006,9022014001004001,9022014003960006,9011014502300000,23,Amhult Resecentrum,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9022014001004001_9011014502300000,9022014003960006_9011014502300000,102.0,9022014001004001,9022014003960006,9022014001004001,9022014003960006,9011014502300000,23,Amhult Resecentrum,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Step 5: Define combinations of day type and time periods for frequency columns
day_type_time_combinations = [
    ('week_day', 'morning'), ('week_day', 'peak_morning'), ('week_day', 'afternoon'), ('week_day', 'peak_evening'),
    ('week_day', 'evening'), ('week_day', 'night'), ('week_day', 'night(dawn)'),
    ('Saturday', 'morning'), ('Saturday', 'peak_morning'), ('Saturday', 'afternoon'), ('Saturday', 'peak_evening'),
    ('Saturday', 'evening'), ('Saturday', 'night'), ('Saturday', 'night(dawn)'),
    ('Sunday', 'morning'), ('Sunday', 'peak_morning'), ('Sunday', 'afternoon'), ('Sunday', 'peak_evening'),
    ('Sunday', 'evening'), ('Sunday', 'night'), ('Sunday', 'night(dawn)')
]

# Step 6: Assign frequencies based on day type and time period combinations
for day, time_period in day_type_time_combinations:
    column_name = f'freq_{day.lower()}_{time_period.lower()}'
    mask = (links_pt_1_long['day_type'] == day) & (links_pt_1_long['time_period'] == time_period)
    links_pt_1_long.loc[mask, column_name] = links_pt_1_long['frequency_daytype']

# Step 7 (optional): Display the results
links_pt_1_long.head()

,source,target,time_distance,stop_id_source,stop_id_target,node_id_source,node_id_target,route_id,route_short_name,stop_name_source,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,9022014001004001_9011014502300000,9022014003960006_9011014502300000,85.0,9022014001004001,9022014003960006,9022014001004001,9022014003960006,9011014502300000,23,Amhult Resecentrum,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9022014001004001_9011014502300000,9022014003960006_9011014502300000,102.0,9022014001004001,9022014003960006,9022014001004001,9022014003960006,9011014502300000,23,Amhult Resecentrum,...,NaN,NaN,NaN,NaN,NaN,NaN,10800.0,NaN,NaN,NaN
2,9022014001004001_9011014502300000,9022014003960006_9011014502300000,85.0,9022014001004001,9022014003960006,9022014001004001,9022014003960006,9011014502300000,23,Amhult Resecentrum,...,NaN,NaN,NaN,NaN,10800.0,NaN,NaN,NaN,NaN,NaN
3,9022014001004001_9011014502300000,9022014003960006_9011014502300000,85.0,9022014001004001,9022014003960006,9022014001004001,9022014003960006,9011014502300000,23,Amhult Resecentrum,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9022014001004001_9011014502300000,9022014003960006_9011014502300000,102.0,9022014001004001,9022014003960006,9022014001004001,9022014003960006,9011014502300000,23,Amhult Resecentrum,...,NaN,NaN,NaN,NaN,NaN,NaN,10800.0,NaN,NaN,NaN


In [13]:
# Aggregate the frequencies by day
# Step 8: Aggregate the frequencies
aggregations = {column: 'sum' for column in links_pt_1_long.columns if column.startswith('freq_')}

links_pt_1_wide = links_pt_1_long.groupby(['source', 'target'], as_index=False).agg({
    **{'time_distance': 'first', 'stop_id_source': 'first', 'stop_id_target': 'first', 'node_id_source': 'first',
       'node_id_target': 'first', 'route_id': 'first', 'route_short_name': 'first', 'stop_name_source': 'first',
       'stop_name_target': 'first', 'place_id_source': 'first', 'place_id_target': 'first', 'mode': 'first',
       'direction_id': 'first', 'stop_sequence': 'first', 'time_period': 'first', 'day_week': 'first',
       'day_type': 'first', 'geometry': 'first'},
    **aggregations
})

# Step 9 (optional): Display the results
links_pt_1_wide.head()

,source,target,time_distance,stop_id_source,stop_id_target,node_id_source,node_id_target,route_id,route_short_name,stop_name_source,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,9022014001004001_9011014502300000,9022014003960006_9011014502300000,85.0,9022014001004001,9022014003960006,9022014001004001,9022014003960006,9011014502300000,23,Amhult Resecentrum,...,0.0,0.0,0.0,0.000000,54000.0,0.000000,54000.000000,0.0,0.000000,0.0
1,9022014001004001_9011014521100000,9022014003960006_9011014521100000,120.0,9022014001004001,9022014003960006,9022014001004001,9022014003960006,9011014521100000,SVART,Amhult Resecentrum,...,0.0,0.0,0.0,4500.000000,4500.0,4500.000000,4500.000000,4500.0,7714.285714,0.0
2,9022014001004001_9011014521200000,9022014003960007_9011014521200000,97.0,9022014001004001,9022014003960007,9022014001004001,9022014003960007,9011014521200000,X1,Amhult Resecentrum,...,0.0,0.0,0.0,2842.105263,3375.0,3176.470588,2842.105263,3600.0,6000.000000,0.0
3,9022014001004003_9011014620400000,9022014006520001_9011014620400000,214.0,9022014001004003,9022014006520001,9022014001004003,9022014006520001,9011014620400000,LILA,Amhult Resecentrum,...,0.0,0.0,0.0,9000.000000,4500.0,6750.000000,4500.000000,10800.0,0.000000,0.0
4,9022014001004004_9011014521200000,9022014006520001_9011014521200000,238.0,9022014001004004,9022014006520001,9022014001004004,9022014006520001,9011014521200000,X1,Amhult Resecentrum,...,0.0,0.0,0.0,3000.000000,3000.0,2842.105263,3176.470588,4500.0,7714.285714,0.0


In [15]:
# Step 10 (optional): Set Geometry and export to the DB
links_pt_1_wide = gpd.GeoDataFrame(links_pt_1_wide, geometry='geometry', crs='3006')
links_pt_1_wide.to_postgis('links_pt_1_wide', engine, schema='pt_1_mdirections_mstops_mroutes', if_exists ='replace')

### Transfers

#### Same stop

In [58]:
# Preprocess the Data
# Step 1: Create a copy of the nodes table
nodes_transfer_same_stop = nodes_pt_1[['node_id', 'stop_id', 'place_id', 'geometry']].copy()

#Step 2 (optional): display the results
nodes_transfer_same_stop.head()

,node_id,stop_id,place_id,geometry
0,9022014001004001_9011014521200000,9022014001004001,307987564,POINT (308137.973 6400982.960)
1,9022014001004003_9011014620400000,9022014001004003,307987564,POINT (308179.982 6400938.925)
2,9022014001004004_9011014521200000,9022014001004004,307987564,POINT (308152.980 6400960.949)
3,9022014001004006_9011014503400000,9022014001004006,307987564,POINT (308158.939 6400936.915)
4,9022014001004007_9011014521100000,9022014001004007,307987564,POINT (308164.962 6400912.990)


In [59]:
# Step 3: Merge the stops in order to create the transfers
# Source
stop_routes_source = nodes_transfer_same_stop.rename(columns={'node_id': 'source', 'stop_id': 'target'}).drop_duplicates()

# Target
stop_routes_target = pd.merge(nodes_transfer_same_stop,
                              nodes_pt_1[['node_id']],
                              on='node_id',
                              how='inner').rename(columns={'stop_id': 'source', 'node_id': 'target'}).drop_duplicates()

# Step 4: Create the transfers same stop table
transfers_same_stop = pd.merge(stop_routes_source, stop_routes_target, how='outer')

#Step 5 (optional): Display the results
transfers_same_stop.head()

,source,target,place_id,geometry
0,9022014001004001,9022014001004001_9011014502300000,307987564,POINT (308137.973 6400982.960)
1,9022014001004001,9022014001004001_9011014521100000,307987564,POINT (308137.973 6400982.960)
2,9022014001004001,9022014001004001_9011014521200000,307987564,POINT (308137.973 6400982.960)
3,9022014001004001_9011014502300000,9022014001004001,307987564,POINT (308137.973 6400982.960)
4,9022014001004001_9011014521100000,9022014001004001,307987564,POINT (308137.973 6400982.960)


In [60]:
# Create the transfers
# Step 6: Merge the table with the frequency of the links table
transfers_same_stop = pd.merge(
    transfers_same_stop,
    links_pt_1_wide[['target'] + [col for col in links_pt_1_wide.columns if col.startswith('freq_')]],
    on='target',
    how='outer'
).drop_duplicates(subset=['source', 'target'])

#Step 7 (optional): Display the results
transfers_same_stop.head()

,source,target,place_id,geometry,freq_monday_morning,freq_tuesday_morning,freq_wednesday_morning,freq_thursday_morning,freq_friday_morning,freq_saturday_morning,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,9022014001004001_9011014502300000,9022014001004001,307987564,POINT (308137.973 6400982.960),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9022014001004001_9011014521100000,9022014001004001,307987564,POINT (308137.973 6400982.960),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9022014001004001_9011014521200000,9022014001004001,307987564,POINT (308137.973 6400982.960),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9022014001004001,9022014001004001_9011014502300000,307987564,POINT (308137.973 6400982.960),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9022014001004001,9022014001004001_9011014521100000,307987564,POINT (308137.973 6400982.960),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
# Step 5: Replace 0 with NaN and vice versa in frequency columns
freq_columns = [col for col in transfers_same_stop_merge.columns if col.startswith('freq_')]
transfers_same_stop_merge[freq_columns] = transfers_same_stop_merge[freq_columns].replace({0: np.nan, np.nan: 0})

In [64]:
# Step 6: Filter final columns for transfers_same_stop table
transfers_same_stop =  transfers_same_stop_merge [[
                                                      'source', 'target', 'geometry']
                                                  + [col for col in transfers_same_stop_merge.columns if col.startswith('freq_')
                                                     ]]
transfers_same_stop.head()

,source,target,geometry,freq_monday_morning,freq_tuesday_morning,freq_wednesday_morning,freq_thursday_morning,freq_friday_morning,freq_saturday_morning,freq_sunday_morning,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,9022014019115003,9022014019115003_9011013061400000,POINT (327675.964 6361431.902),0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,27971.223022,0.0,27000.0,54000.0,0.0,0.0
25,9022014019610001,9022014019610001_9011013061400000,POINT (327108.979 6359778.001),0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,27971.223022,0.0,27000.0,54000.0,0.0,0.0
50,9022014019610002,9022014019610002_9011013061400000,POINT (327161.986 6359701.973),0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,27971.223022,0.0,27000.0,54000.0,0.0,0.0
75,9022014019611001,9022014019611001_9011013061400000,POINT (326796.970 6360275.948),0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,27971.223022,0.0,27000.0,54000.0,0.0,0.0
100,9022014019611002,9022014019611002_9011013061400000,POINT (326818.972 6360170.911),0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,27971.223022,0.0,27000.0,54000.0,0.0,0.0


In [66]:
# Step 7 (optional): Set Geometry and export the table to the DB
transfers_same_stop = gpd.GeoDataFrame(transfers_same_stop, geometry='geometry', crs='3006')
transfers_same_stop.to_postgis('transfers_same_stop_pt_1_long', engine, schema='pt_1_mdirections_mstops_mroutes', if_exists ='replace')

#### Different stops

In [74]:
#Preprocess the Data
# Step 1: Create a copy of the nodes table
nodes_transfer_diff_stop = nodes_pt_1[['node_id', 'stop_id', 'geometry']].copy()

# Step 2 (optional): Display the results
nodes_transfer_diff_stop.head()

,node_id,stop_id,geometry
0,9022014001004001_9011014521200000,9022014001004001,POINT (308137.973 6400982.960)
1,9022014001004003_9011014620400000,9022014001004003,POINT (308179.982 6400938.925)
2,9022014001004004_9011014521200000,9022014001004004,POINT (308152.980 6400960.949)
3,9022014001004006_9011014503400000,9022014001004006,POINT (308158.939 6400936.915)
4,9022014001004007_9011014521100000,9022014001004007,POINT (308164.962 6400912.990)


In [75]:
#Create the buffers
# Step 3: Create a buffer column and set it as geometry
nodes_transfer_diff_stop['buffer'] = nodes_transfer_diff_stop['geometry'].buffer(85)
nodes_transfer_diff_stop = nodes_transfer_diff_stop.set_geometry('buffer',
                                                                 crs='EPSG:3006')

# Step 4: found the intersections among the buffers
nodes_transfer_diff_stop = gpd.overlay(nodes_transfer_diff_stop,
                                       nodes_transfer_diff_stop,
                                       how='intersection').rename(columns={'node_id_1':'source',
                                                                           'node_id_2':'target',
                                                                           'stop_id_1':'stop_id_source',
                                                                           'stop_id_2':'stop_id_target',
                                                                           'geometry_1': 'geometry_source',
                                                                           'geometry_2': 'geometry_target'}).dropna(subset=['source'])
# Step 5 (optional): Display the results
nodes_transfer_diff_stop.head()

,source,stop_id_source,geometry_source,target,stop_id_target,geometry_target,geometry
0,9022014001004001_9011014521200000,9022014001004001,POINT (308137.973 6400982.960),9022014001004001_9011014521200000,9022014001004001,POINT (308137.973 6400982.960),"POLYGON ((308212.611 6400975.609, 308211.531 6..."
1,9022014001004001_9011014521200000,9022014001004001,POINT (308137.973 6400982.960),9022014001004003_9011014620400000,9022014001004003,POINT (308179.982 6400938.925),"POLYGON ((308212.611 6400975.609, 308211.531 6..."
2,9022014001004001_9011014521200000,9022014001004001,POINT (308137.973 6400982.960),9022014001004004_9011014521200000,9022014001004004,POINT (308152.980 6400960.949),"POLYGON ((308212.611 6400975.609, 308211.531 6..."
3,9022014001004001_9011014521200000,9022014001004001,POINT (308137.973 6400982.960),9022014001004006_9011014503400000,9022014001004006,POINT (308158.939 6400936.915),"POLYGON ((308212.611 6400975.609, 308211.531 6..."
4,9022014001004001_9011014521200000,9022014001004001,POINT (308137.973 6400982.960),9022014001004007_9011014521100000,9022014001004007,POINT (308164.962 6400912.990),"POLYGON ((308211.531 6400968.328, 308209.743 6..."


In [76]:
#Create the geometry
# Step 6: Create linstrings between the intersections
nodes_transfer_diff_stop['geometry'] = nodes_transfer_diff_stop.apply(lambda row: LineString([[row.geometry_source.x,
                                                                                               row.geometry_source.y],
                                                                                                [row.geometry_target.x,
                                                                                                 row.geometry_target.y]]) if row.geometry_target is not None else None,
                                                                      axis=1)
# Step 7: Set the length of the linestring
nodes_transfer_diff_stop['length'] = nodes_transfer_diff_stop['geometry'].length
nodes_transfer_diff_stop['time_distance'] = nodes_transfer_diff_stop['length'] / 1.2

# Step 8 (optional): Display the results
nodes_transfer_diff_stop.head()

,source,stop_id_source,geometry_source,target,stop_id_target,geometry_target,geometry,length,time_distance
0,9022014001004001_9011014521200000,9022014001004001,POINT (308137.973 6400982.960),9022014001004001_9011014521200000,9022014001004001,POINT (308137.973 6400982.960),"LINESTRING (308137.973 6400982.960, 308137.973...",0.000000,0.000000
1,9022014001004001_9011014521200000,9022014001004001,POINT (308137.973 6400982.960),9022014001004003_9011014620400000,9022014001004003,POINT (308179.982 6400938.925),"LINESTRING (308137.973 6400982.960, 308179.982...",60.859819,50.716516
2,9022014001004001_9011014521200000,9022014001004001,POINT (308137.973 6400982.960),9022014001004004_9011014521200000,9022014001004004,POINT (308152.980 6400960.949),"LINESTRING (308137.973 6400982.960, 308152.980...",26.640277,22.200231
3,9022014001004001_9011014521200000,9022014001004001,POINT (308137.973 6400982.960),9022014001004006_9011014503400000,9022014001004006,POINT (308158.939 6400936.915),"LINESTRING (308137.973 6400982.960, 308158.939...",50.593603,42.161336
4,9022014001004001_9011014521200000,9022014001004001,POINT (308137.973 6400982.960),9022014001004007_9011014521100000,9022014001004007,POINT (308164.962 6400912.990),"LINESTRING (308137.973 6400982.960, 308164.962...",74.995341,62.496118


In [77]:
#Filter and organize the tables
# Step 9: Filter the transfers between different stops
transfers_diff_stop = nodes_transfer_diff_stop[nodes_transfer_diff_stop['length'] > 0]

# Step 10: Drop the duplicates of symmetric pairs
transfers_diff_stop ['symmetric_pairs'] = transfers_diff_stop[['source',
                                                               'target']].apply(lambda x: '-'.join(sorted(x)),
                                                                                axis=1)
transfers_diff_stop = transfers_diff_stop.drop_duplicates(subset = ['symmetric_pairs'])

# Step 11: Filter the necessary columns
transfers_diff_stop = transfers_diff_stop [['stop_id_source',
                                            'stop_id_target',
                                            'geometry',
                                            'length',
                                            'time_distance']].rename(columns={'stop_id_source':'source',
                                                                              'stop_id_target':'target'}).drop_duplicates()
# Step 12 (optional): Display the results
transfers_diff_stop.head()

C:\Users\monaliza\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,source,target,geometry,length,time_distance
1,9022014001004001,9022014001004003,"LINESTRING (308137.973 6400982.960, 308179.982...",60.859819,50.716516
2,9022014001004001,9022014001004004,"LINESTRING (308137.973 6400982.960, 308152.980...",26.640277,22.200231
3,9022014001004001,9022014001004006,"LINESTRING (308137.973 6400982.960, 308158.939...",50.593603,42.161336
4,9022014001004001,9022014001004007,"LINESTRING (308137.973 6400982.960, 308164.962...",74.995341,62.496118
10,9022014001004003,9022014001004004,"LINESTRING (308179.982 6400938.925, 308152.980...",34.845377,29.037814


In [78]:
transfers_diff_stop

,source,target,geometry,length,time_distance
1,9022014001004001,9022014001004003,"LINESTRING (308137.973 6400982.960, 308179.982...",60.859819,50.716516
2,9022014001004001,9022014001004004,"LINESTRING (308137.973 6400982.960, 308152.980...",26.640277,22.200231
3,9022014001004001,9022014001004006,"LINESTRING (308137.973 6400982.960, 308158.939...",50.593603,42.161336
4,9022014001004001,9022014001004007,"LINESTRING (308137.973 6400982.960, 308164.962...",74.995341,62.496118
10,9022014001004003,9022014001004004,"LINESTRING (308179.982 6400938.925, 308152.980...",34.845377,29.037814
...,...,...,...,...,...
344256,9022014052508001,9022014052508002,"LINESTRING (417687.984 6388737.970, 417689.945...",23.094829,19.245691
344280,9022014052612004,9022014052612006,"LINESTRING (401194.940 6372618.918, 401161.944...",44.552924,37.127436
344714,9022014075045005,9022014075045008,"LINESTRING (481717.961 6390073.947, 481750.974...",72.058945,60.049121
345923,9022014082017014,9022014082900004,"LINESTRING (376765.998 6399417.991, 376750.968...",71.569168,59.640974


In [50]:
# Step 6 (optional): Set Geometry and export the table to the DB
transfers_diff_stop = gpd.GeoDataFrame(transfers_diff_stop, geometry='geometry', crs='3006')
transfers_diff_stop.to_postgis('transfers_diff_stop_pt_1_long', engine, schema='pt_1_mdirections_mstops_mroutes', if_exists ='replace')